In [7]:
def read_json():
  for i in range(10):
    temp = reader.read()
    exp = 0
    # If there is no short answer, only add example to training set with 5% probability
    if temp["annotations"][0]["short_answers"] == []:
      if random.uniform(0, 1) > .9:
        exp = Example(temp)
        if i < 100:
          examples.append(exp)
        for a in range(exp.examples.shape[0]):
          yield (exp.examples[a, :], exp.labels[a, :], exp.token_type_ids[0, :])
    else:
      exp = Example(temp)
      if i < 100:
        examples.append(exp)
      for a in range(exp.examples.shape[0]):
        yield (exp.examples[a, :], exp.labels[a, :], exp.token_type_ids[0, :])
        
        
reader = jsonlines.open("v1-train.jsonl")

In [13]:
import jsonlines
import json
import copy

Each data will contain the question, the label (whether there is a long answer or not), the original text, and the id.

In [16]:
new_data =  {
        'id': 0,
        'url': None,
        'question_text': None,
        'original_text': None,
        'answer': False,
        'answer_loc': [-1, -1]
    }

data = {'new_data':[]}

i = 0
j = 0

while i+j < 50000:
    new = copy.deepcopy(new_data)
    data_point = reader.read()
    if data_point["annotations"][0]["long_answer"]["start_token"] != -1 and i < 25000:
        new['original_text'] = data_point["document_text"]
        new['url'] = data_point["document_url"]
        new['id'] = data_point["example_id"]
        new['answer'] = (data_point["annotations"][0]["long_answer"]["start_token"] != -1)
        new['question_text'] = data_point["question_text"]
        if new['answer'] == True:
            new['answer_loc'] = [data_point["annotations"][0]["long_answer"]["start_token"], data_point["annotations"][0]["long_answer"]["end_token"]]
        data['new_data'].append(new)
        i += 1
        
    elif data_point["annotations"][0]["long_answer"]["start_token"] == -1 and j < 25000:
        new['original_text'] = data_point["document_text"]
        new['url'] = data_point["document_url"]
        new['id'] = data_point["example_id"]
        new['answer'] = (data_point["annotations"][0]["long_answer"]["start_token"] != -1)
        new['question_text'] = data_point["question_text"]
        if new['answer'] == True:
            new['answer_loc'] = [data_point["annotations"][0]["long_answer"]["start_token"], data_point["annotations"][0]["long_answer"]["end_token"]]
        data['new_data'].append(new)
        j += 1



In [17]:
#Download dataset
with open('NQAltered.json', 'w') as outfile:
    json.dump(data, outfile)